In [ ]:
# Copyright 2025 50Hertz Transmission GmbH and Elia Transmission Belgium
#
# This Source Code Form is subject to the terms of the Mozilla Public License, v. 2.0.
# If a copy of the MPL was not distributed with this file,
# you can obtain one at https://mozilla.org/MPL/2.0/.
# Mozilla Public License, version 2.0

## Interactive MAP-Elites postprocess and filtering

### What is this ?

Use this notebook to filter the topologies based on their fitness and metrics. This notebook then regenerates a new res.json file.

### How-to steps : 
1. Edit the variables in the __Variables and Loading cell__
2. Run the __Functions__ cell
3. Run the __Interactive Filtering__ cell
3. Run the __Save to Disk__ cell

### 1. Variables and Loading

In [ ]:
import json
import os

# Variables
FOLDER = "/workspaces/AICoE_HPC_RL_Optimizer/stats"
FILE_NAME = "res_tests.json"
OUTPUT_NAME = "filtered_topologies.json"

# Load res.json data
with open(os.path.join(FOLDER, FILE_NAME), 'r') as file:
    data = json.load(file)

### 2. Functions

In [ ]:
import math
import ipywidgets as widgets
from IPython.display import display
import numpy as np


global min_max_metrics

test_data = {
    "best_topos": [
        {
            "metrics": {
                "metric_test1" : 1.
            },
            "fitness" : 2.,
        }
    ],
}

# get the min an max value of each metric from the topologies
def get_min_max_values(data: dict) -> dict:
    """Get the minimum and maximum values for each metric in the topologies."""
    observed_metrics = data["best_topos"][0]["metrics"].keys()
    min_max = {}
    for metric in observed_metrics:
        min_max[metric] = {
            "min": np.inf,
            "max": -np.inf
        }
        for topo in data["best_topos"]:
            if topo["metrics"][metric] < min_max[metric]["min"]:
                min_max[metric]["min"] = topo["metrics"][metric]
            if topo["metrics"][metric] > min_max[metric]["max"]:
                min_max[metric]["max"] = topo["metrics"][metric]
    return min_max

min_max_test = get_min_max_values(test_data)
assert min_max_test == {
    "metric_test1": {
        "min": 1.0,
        "max": 1.0
    }
}


def add_fitness_to_metrics(data: dict) -> dict:
    """artificially add fitness to metrics for convenience"""
    data_augmented = data.copy()
    for topo in data_augmented["best_topos"]:
        topo["metrics"]["fitness"] = topo["fitness"]
    return data_augmented

test_data_augmented = add_fitness_to_metrics(test_data)
assert "fitness" in test_data_augmented["best_topos"][0]["metrics"]


def filter_topos(data: dict, min_max_metrics: dict) -> dict:
    """Only keeps the topologies whose metrics are between the limits"""
    filtered_topos = []
    for topo in data["best_topos"]:
        keep = True
        for metric, values in min_max_metrics.items():
            if topo["metrics"][metric] < values["min"] or topo["metrics"][metric] > values["max"]:
                keep = False
                break
        if keep:
            filtered_topos.append(topo)
    data_filtered = data.copy()
    data_filtered["best_topos"] = filtered_topos
    return data_filtered

test_data_filtered = filter_topos(test_data, get_min_max_values(test_data))
assert test_data_filtered == test_data
test_min_max_metrics2 = {"metric_test1": {"min": 0.0,"max": 0.0}}
test_data_filtered2 = filter_topos(test_data, test_min_max_metrics2)
assert test_data_filtered2 == {"best_topos": []}


def plot_sliders(data_augmented: dict):
    """For each metric, plots a slider to select the range of values to keep."""

    for metric in data_augmented["best_topos"][0]["metrics"]:
        # Define a list of arbitrary values
        value_list = []
        for topo in data_augmented["best_topos"]:
            value_list.append(topo["metrics"][metric])
        
        # Create two SelectionSliders
        min_slider = widgets.SelectionSlider(
            options=sorted(value_list),
            value=min(value_list),  # Default minimum value
            description='Min:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True
        )
        max_slider = widgets.SelectionSlider(
            options=sorted(value_list),
            value=max(value_list),  # Default maximum value
            description='Max:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True
        )
        
        # Create a label for the sliders
        title_label = widgets.Label(value=f"{metric} range :")
        
        # Function to display the selected min/max values
        def on_value_change(change, metric=metric, min_slider=min_slider, max_slider=max_slider):
            # Declare min_max_metrics as nonlocal to modify the outer scope variable
            min_max_metrics[metric]["min"] = min_slider.value
            min_max_metrics[metric]["max"] = max_slider.value
            filtered_data = filter_topos(data_augmented, min_max_metrics)
            msg = f"{len(filtered_data['best_topos'])} topologies remaining."  # {metric} range: {min_slider.value} to {max_slider.value}.
            print(f'\r{msg}{" " * 50}', end='')

        # Attach the function to both sliders' value changes
        min_slider.observe(on_value_change, names="value")
        max_slider.observe(on_value_change, names="value")

        # Display the title and both sliders in a vertical box
        display(widgets.VBox([title_label, widgets.HBox([min_slider, max_slider])]))


def recreate_res_json(data: dict, min_max_metrics: dict) -> dict:
    """Recreate the res.json file with the filtered topologies. 
    Also removes the fitness from the metrics."""
    filtered_data = filter_topos(data, min_max_metrics)
    for topo in filtered_data["best_topos"]:
        topo["fitness"] = topo["metrics"]["fitness"]
        del topo["metrics"]["fitness"]
    return filtered_data

test_recreate_data = recreate_res_json(test_data_augmented, get_min_max_values(test_data))
assert math.isclose(test_recreate_data["best_topos"][0]["fitness"], 2.0)
assert "fitness" not in test_recreate_data["best_topos"][0]["metrics"]


def save_filtered_topos(data: dict):
    """Save the filtered topologies to a JSON file."""
    path = os.path.join(FOLDER, OUTPUT_NAME)
    with open(path, 'w') as f:
        json.dump(data, f, indent=4)
    print(f"Filtered JSON saved to {path}")

### 3. Interactive filtering

In [ ]:
data_augmented = add_fitness_to_metrics(data)
min_max_metrics = get_min_max_values(data_augmented)
plot_sliders(data_augmented)

### 4. Save to Disk

In [ ]:
data_filtered = recreate_res_json(data_augmented, min_max_metrics)
save_filtered_topos(data_filtered)